# BRONZE TABLES CREATION

In [2]:
%%pyspark

orders_raw = spark.read.parquet("abfss://be300475-9088-41d0-a453-ffe3be53ada6@onelake.dfs.fabric.microsoft.com/efe7fcf8-4531-4d21-987d-ad6141c39568/Files/bronze/orders.parquet")
customers_raw = spark.read.parquet("abfss://be300475-9088-41d0-a453-ffe3be53ada6@onelake.dfs.fabric.microsoft.com/efe7fcf8-4531-4d21-987d-ad6141c39568/Files/bronze/customers.parquet")
products_raw = spark.read.parquet("abfss://be300475-9088-41d0-a453-ffe3be53ada6@onelake.dfs.fabric.microsoft.com/efe7fcf8-4531-4d21-987d-ad6141c39568/Files/bronze/products.parquet")
order_items_raw = spark.read.parquet("abfss://be300475-9088-41d0-a453-ffe3be53ada6@onelake.dfs.fabric.microsoft.com/efe7fcf8-4531-4d21-987d-ad6141c39568/Files/bronze/order_items.parquet")

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 4, Finished, Available, Finished)

In [5]:
%%pyspark
orders_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_orders")
customers_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_customers")
products_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_products")
order_items_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_order_items")

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 7, Finished, Available, Finished)

In [6]:
select * from bronze_customers

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 8, Finished, Available, Finished)

<Spark SQL result set with 25 rows and 4 fields>

In [7]:
select * from bronze_order_items limit 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 9, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 3 fields>

In [8]:
select * from bronze_orders limit 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 10, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 9 fields>

In [9]:
select * from bronze_products limit 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 11, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 5 fields>

# SILVER TABLE

## silver_orders

In [11]:
SELECT * FROM  bronze_orders LIMIT 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 13, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 9 fields>

In [12]:
CREATE OR REPLACE TABLE silver_orders AS
SELECT
  CAST(OrderID AS STRING) AS OrderID,
  CAST(CustomerID AS STRING) AS CustomerID,
  TO_DATE(REPLACE(OrderDate, '/', '-')) AS OrderDate,
  TO_DATE(REPLACE(ShippedDate, '/', '-')) AS ShippedDate,
  TO_DATE(REPLACE(DeliveredDate, '/', '-')) AS DeliveredDate,
  ROUND(CAST(Price AS DOUBLE), 2) AS Price,
  INITCAP(TRIM(OrderChannel)) AS OrderChannel,
  INITCAP(TRIM(Region)) AS Region,
  TRIM(ReferredBy) AS ReferredBy
FROM bronze_orders
WHERE OrderID IS NOT NULL;


StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 14, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
SELECT * FROM silver_orders LIMIT 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 16, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 9 fields>

# SILVER CUSTOMER

In [15]:
SELECT * FROM  bronze_customers LIMIT 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 17, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>

In [16]:
CREATE OR REPLACE TABLE silver_customers AS
SELECT
  CAST(CustomerID AS STRING) AS CustomerID,
  INITCAP(TRIM(CustomerName)) AS CustomerName,
  CASE 
    WHEN LOWER(IsPremiumCustomer) IN ('yes', 'y') THEN 'Yes'
    ELSE 'No'
  END AS IsPremiumCustomer,
  TO_DATE(SignupDate) AS SignupDate
FROM bronze_customers
WHERE CustomerID IS NOT NULL;

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [17]:
SELECT * FROM silver_customers LIMIT 5

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 19, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 4 fields>

# SILVER PRODUCTS

In [20]:
SELECT * FROM  bronze_products LIMIT 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 22, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 5 fields>

In [21]:
CREATE OR REPLACE TABLE silver_products AS
SELECT
  CAST(ProductID AS STRING) AS ProductID,
  INITCAP(TRIM(Category)) AS Category,
  INITCAP(TRIM(Carrier)) AS Carrier,
  ROUND(CAST(ShipmentCost AS DOUBLE), 2) AS ShipmentCost,
  CASE 
    WHEN LOWER(IsDiscounted) IN ('yes', 'y') THEN 'Yes'
    ELSE 'No'
  END AS IsDiscounted
FROM bronze_products
WHERE ProductID IS NOT NULL;

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 23, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [23]:
SELECT * FROM silver_products LIMIT 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 25, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 5 fields>

# SILVER ORDERS 

In [24]:
SELECT * FROM bronze_order_items LIMIT 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 26, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 3 fields>

In [25]:
CREATE OR REPLACE TABLE silver_order_items AS
SELECT
  CAST(OrderID AS STRING) AS OrderID,
  CAST(ProductID AS STRING) AS ProductID,
  CAST(Quantity AS INT) AS Quantity
FROM bronze_order_items
WHERE Quantity IS NOT NULL AND Quantity > 0;

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 27, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [26]:
SELECT * FROM silver_order_items LIMIT 4

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 28, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 3 fields>

# GOLD 

In [27]:
CREATE OR REPLACE TEMP VIEW orders_enriched AS
SELECT
  o.OrderID,
  o.CustomerID,
  o.OrderDate,
  o.ShippedDate,
  o.DeliveredDate,
  o.Price,
  o.Region,
  o.OrderChannel,
  o.ReferredBy,
  c.CustomerName,
  c.IsPremiumCustomer,
  c.SignupDate,
  i.ProductID,
  i.Quantity,
  p.Category,
  p.Carrier,
  p.ShipmentCost,
  p.IsDiscounted,
  DATEDIFF(o.DeliveredDate, o.ShippedDate) AS DeliveryDelay
FROM silver_orders o
LEFT JOIN silver_customers c ON o.CustomerID = c.CustomerID
LEFT JOIN silver_order_items i ON o.OrderID = i.OrderID
LEFT JOIN silver_products p ON i.ProductID = p.ProductID;


StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 29, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [28]:
select * from orders_enriched

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 30, Finished, Available, Finished)

<Spark SQL result set with 37 rows and 19 fields>

In [29]:
CREATE OR REPLACE TABLE gold_order_kpis AS
SELECT
  CustomerID,
  CustomerName,
  IsPremiumCustomer,
  Region,
  OrderChannel,
  COUNT(DISTINCT OrderID) AS TotalOrders,
  SUM(Quantity) AS TotalQuantity,
  SUM(Price) AS TotalRevenue,
  ROUND(AVG(DeliveryDelay), 1) AS AvgDeliveryDelayDays,
  COUNT(DISTINCT CASE WHEN DeliveryDelay > 3 THEN OrderID END) AS DelayedOrders,
  ROUND(SUM(ShipmentCost), 2) AS TotalShipmentCost,
  ROUND(SUM(CASE WHEN IsDiscounted = 'Yes' THEN Price ELSE 0 END), 2) AS RevenueFromDiscounted,
  MAX(DeliveryDelay) AS MaxDeliveryDelay
FROM orders_enriched
GROUP BY 
  CustomerID, CustomerName, IsPremiumCustomer, Region, OrderChannel;


StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 31, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [30]:
select * from gold_order_kpis

StatementMeta(, ae2e62ec-5575-4d00-9cc7-d5ea03addb71, 32, Finished, Available, Finished)

<Spark SQL result set with 25 rows and 13 fields>